## StarTracker WCS solutions - 31-Jan-23

Craig Lage - Reworked to work with the new summit_utils code

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION

import lsst.afw.image as afwImage
from lsst.geom import SpherePoint
from lsst.geom import Angle as afwAngle
from astroquery.simbad import Simbad

from lsst.summit.utils.utils import starTrackerFileToExposure
from lsst.summit.utils.astrometry.anet import CommandLineSolver
from lsst.summit.utils.astrometry.utils import runCharactierizeImage

In [ ]:
solver = CommandLineSolver('/project/shared/ref_cats/astrometry_net/')

In [ ]:
#Enter which image you want to look at
#[camera, num] = ['Narrow', 102]
[camera, num] = ['Wide', 101]
date = 20221128
seqNum = 100


year = int(date/10000)
month = int((date - 10000 * year)/100)
day = int((date - 10000 * year - 100 * month))

path = f"/project/GenericCamera/{num}/{year}/{month:02}/{day:02}/"
print(path)
filename = path + f"GC{num}_O_{date}_{seqNum:06}.fits"
exp = starTrackerFileToExposure(filename)
results = runCharactierizeImage(exp, 5, 25)
if camera == 'Narrow':
    # Need to loosen percentageScaleError because header SECPIX is wrong
    solverResults = solver.run(exp, results.sourceCat, percentageScaleError=60.0, isWideField=False)
if camera == 'Wide':
    solverResults = solver.run(exp, results.sourceCat, isWideField=True)
newWcs = solverResults.wcs
rms_error = solverResults.rmsErrorArsec
raSolve = newWcs.getSkyOrigin().getRa().asDegrees()
decSolve = newWcs.getSkyOrigin().getDec().asDegrees()

hdulist = pf.open(filename)
header = hdulist[0].header
raStart = header["RASTART"]
decStart = header["DECSTART"]
azStart = header["AZSTART"]
#azStart = 360.0 - azStart # Compensating for Az sign error
if azStart > 360.0:
    azStart -= 360.0
if azStart < 0.0:
    azStart += 360.0

elStart = header["ELSTART"]
rotpa = header["ROTPA"]
rot = newWcs.getRelativeRotationToWcs(exp.getWcs()).asDegrees()
time = Time(header["DATE-OBS"], scale='tai')
altAz = AltAz(obstime=time, location=SIMONYI_LOCATION)
skyLocation = SkyCoord(raSolve * u.deg, decSolve * u.deg)
obsAltAz = skyLocation.transform_to(altAz)
az = obsAltAz.az.deg
el = obsAltAz.alt.deg
deltaAz = azStart - az
if deltaAz > 360.0:
    deltaAz -= 360.0
if deltaAz < 0.0:
    deltaAz += 360.0
deltaEl = elStart - el
print(f"{camera}, 202212{date:02}_{seqNum}, {deltaAz:.5f}, {deltaEl:.5f}")

# Now find objects within the image
if camera == 'Narrow':
    radius = 0.5
    mag_range = 9.5
if camera == 'Wide':
    radius = 3.0
    mag_range = 7.5
mag_limit = 0.0
criteria = (f"region(circle,ICRS,{raSolve} {decSolve},{radius}d) & "
            f"Vmag > {mag_limit} & Vmag < {mag_limit+mag_range} & "
            "cat = HD")
results = Simbad.query_criteria(criteria)
#print(results)
xs = []
ys = []
names = []
for i in range(len(results)):
    ra = Angle(results[i][1] + 'hours')
    dec = Angle(results[i][2] + 'degrees')
    spherePoint = SpherePoint(afwAngle(ra.rad), afwAngle(dec.rad))
    point = newWcs.skyToPixel(spherePoint)
    xs.append(point.x)
    ys.append(point.y)
    name = results[i][0]
    names.append(name)

In [ ]:
# Now plot the data with matplotlib
%matplotlib inline
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

plt.figure(figsize=(16,16))
plt.title(f"StarTracker {camera}")
img = plt.imshow(exp.image.array,  interpolation='Nearest', cmap='gray', vmin=1000, vmax=10000)
plt.scatter(xs, ys\
            ,facecolors='none', edgecolors='g', s=200, lw=2)
for x, y, name in zip(xs, ys, names):
    plt.text(x+50,y, name, color='g')
colorbar(img)
plt.tight_layout(h_pad=1)
plt.savefig(f"/home/craiglagegit/DATA/StarTracker_Labels_Simbad_{camera}_{date}_{seqNum}.pdf")

In [ ]:
newWcs.getPixelScale